In [ ]:
#Loading Libraries
%conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
%pip install -U adapter-transformers
%conda install -y -c conda-forge tensorboard
%pip install optuna

In [ ]:
from datasets import load_dataset

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

In [ ]:
scierc_dataset['train'][255]

This block creates dataset for TAPT

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch_pretraining(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

# Encode the input data
# NOTE: num_proc does not seem to work, for some reason it can't find the tokenizer
scierc_dataset_pretraining = scierc_dataset.map(encode_batch_pretraining, 
                                    batched=True, 
                                    remove_columns=scierc_dataset['train'].column_names, 
                                    )

def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples
  
scierc_dataset_pretraining = scierc_dataset_pretraining.map(add_labels, batched=True)
scierc_dataset_pretraining.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



In [ ]:
# Collater adds padding in the form of EOS tokens, makes data augmentations of random masking ('mlm_probability)
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Here we are creating the dataset for task finetuning

In [ ]:
# Finding the number of labels
import numpy as np
labels = np.unique(np.array(scierc_dataset['train']['label']))
num_of_labels = labels.size

print(labels)
print(num_of_labels)

In [ ]:
# encoding the labels
def encode_labels(dataset):
    for i in range(num_of_labels):
        if dataset['label'] == labels[i]:
            dataset['label'] = i
    return dataset

scierc_dataset = scierc_dataset.map(encode_labels)
scierc_dataset['train'][0]

In [ ]:
def encode_batch_finetuning(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
scierc_dataset_finetuning = scierc_dataset.map(encode_batch_finetuning, batched=True)
# The transformers model expects the target class column to be named "labels"
scierc_dataset_finetuning = scierc_dataset_finetuning.rename_column("label", 'labels')
# Transform to pytorch tensors and only output the required columns
scierc_dataset_finetuning.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Model Creation

In [ ]:
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

def model_init(adapter_name = 'default_adapter', 
               num_lables = 0, 
               pretraining = False,
               load_adapter = False):
    
    if pretraining:
        config = RobertaConfig.from_pretrained(
            "roberta-base",
            # num_labels=num_of_labels,
        )
        model = RobertaAdapterModel.from_pretrained(
            "roberta-base",
            config=config,
        )
        if load_adapter:
            # Add new adapter
            model.load_adapter(adapter_name)

        else:
            # Add new adapter
            model.add_adapter(adapter_name)
            
        # Add a matching classification head
        model.add_masked_lm_head(adapter_name)
            
    else:
        config = RobertaConfig.from_pretrained(
            "roberta-base",
            num_labels=num_of_labels,
        )
        model = RobertaAdapterModel.from_pretrained(
            "roberta-base",
            config=config,
        )
        
        if load_adapter:
            # Add new adapter
            model.load_adapter(adapter_name)

        else:
            # Add new adapter
            model.add_adapter(adapter_name)
            
        # Add a matching classification head
        model.add_classification_head(
                adapter_name,
                num_labels=num_lables,
                id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
                        3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
                )
            
    # Activate the adapter
    model.train_adapter(adapter_name)    
     
    return model

Pretraining Block

In [ ]:
from transformers import TrainingArguments, AdapterTrainer
from datasets import load_metric
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback

def pretraining_loop(num_models, training_args, dataset, 
                     data_collator, adapter_name, DAPT_n_TAPT, 
                     TAPT_dataset):

    for i in range(num_models):
        adapter_name = f"{adapter_name}_{i}"
        model = model_init(adapter_name = adapter_name, pretraining=True)
        
        writer = SummaryWriter()
        writer = TensorBoardCallback(writer, filename_suffix = adapter_name)

        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            data_collator=data_collator,  
            callbacks=[writer] 
        )
        
        trainer.train()
        
        f = open("evaulations.txt", "a")
        f.write(adapter_name)
        f.write(trainer.evaluate(dataset['test']))
        f.write('\n')
        f.close()
        
        model.save_pretrained(f"{adapter_name}")
        
        # if DAPT_n_TAPT:
        #     trainer = AdapterTrainer(
        #         model=model,
        #         args=training_args,
        #         train_dataset=TAPT_dataset["train"],
        #         eval_dataset=TAPT_dataset["validation"],
        #         data_collator=data_collator,  
        #         callbacks=[writer] 
        #     )
            
        #     trainer.train()
        
        #     f = open("DAPT_TAPT_evaulations.txt", "a")
        #     f.write(adapter_name)
        #     f.write(trainer.evaluate(TAPT_dataset['test']))
        #     f.write('\n')
        #     f.close()
            
        #     model.save_pretrained(f"{adapter_name}_DAPT_TAPT")

DAPT Training

In [ ]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/pretraining/DAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
pretraining_loop(num_models = 5, 
                 training_args = training_args, 
                #  dataset = DAPT_dataset, TODO: Need to add DAPT training set
                 data_collator = data_collator, 
                 adapter_name = "DAPT_sci-erc")

DAPT+TAPT Training

TAPT Training

In [ ]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/pretraining/TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
pretraining_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_pretraining, 
                 data_collator = data_collator, 
                 adapter_name = "TAPT_sci-erc")

Fine Tuning Models

In [ ]:
def finetuning_loop(num_models, training_args, dataset, adapter_name, load_adapter = False):

    for i in range(num_models):
        adapter_name = f"{adapter_name}_{i}"
        model = model_init(adapter_name = adapter_name, pretraining=False, load_adapter = load_adapter)
        
        writer = SummaryWriter()
        writer = TensorBoardCallback(writer, filename_suffix = adapter_name)

        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            data_collator=data_collator,  
            callbacks=[writer] 
        )
        
        trainer.train()
        
        f = open("evaulations.txt", "a")
        f.write(adapter_name)
        f.write(trainer.evaluate(dataset['test']))
        f.write('\n')
        f.close()
        
        model.save_pretrained(f"{adapter_name}")

DAPT Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/finetuning/DAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "DAPT_sci-erc",
                 load_adapter = True)

DAPT+TAPT Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/finetuning/DAPT_TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "DAPT_TAPT_sci-erc",
                 load_adapter = True)

TAPT Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/finetuning/TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "TAPT_sci-erc",
                 load_adapter = True)

Only Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/finetuning/No_Pretrain",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "sci-erc",
                 load_adapter = False)